# EM2

par Haolin PAN et Shikang DU

In [1]:
# import les packages
import numpy as np
import pandas as pd
import scipy.stats as sps
import math
import matplotlib.pyplot as plt

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, gridplot
output_notebook(hide_banner=True)

# set matplotlib
%matplotlib inline

## Question 1  Vecteurs gaussiens

Comme $4u \sim \chi^2(2)$, alors on a $u \sim \dfrac{\chi^2(2)}{4}$. 

In [2]:
# Simulation de loi chi
df = 2
N = 1000

chi = sps.chi(2,df)
sim_U = np.array(chi.rvs(N))
for item in sim_U:
    item = item/4
# print(sim_U)

x = np.linspace(chi.ppf(0.001),chi.ppf(0.999), 100)
rv = chi.pdf(x)

#Visualization de Simulation

# code de PLT 

# plt.hist(sim_U)
# plt.title('Simulation de u')
# plt.count

fig1 = figure(title='Simulation de U', width=490, height=300, background_fill_color="#fafafa")
hist, edges = np.histogram(sim_U, bins=20)
fig1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], 
          fill_color="navy", line_color="white", alpha=0.5, legend="simulation")
fig1.line(x, rv*N/4, line_color="#ff8888", line_width=4, alpha=0.7, legend="PDF * N / 4")
fig1.yaxis.axis_label = 'count'
fig1.xaxis.axis_label = 'U'
fig1.grid.grid_line_color="white"
show(fig1)

## Question 2 Vecteur Gaussien à Dimension 2

In [3]:
# Simulation de y1 et y2
y1 = sps.norm(0,1)
y2 = sps.norm(0,1)
sim_Y1 = np.array(y1.rvs(1000))
sim_Y2 = np.array(y2.rvs(1000))

sim_Y = []
for i in range(len(sim_Y1)):
  sim_Y.append(np.array([sim_Y1[i], sim_Y2[i]]))

#Transaction par matrix
sigma = np.array([ [0.535, 0.064], [0.064, 0.097] ])

sim_Z = []
for i in range(len(sim_Y)):
    sim_Z.append(np.array([1,2]) + np.dot(sigma, sim_Y[i]))
print(sim_Z)

[array([0.19561964, 1.96620501]), array([0.75226338, 2.09511078]), array([0.39002003, 2.01072886]), array([0.25577489, 1.96085716]), array([1.12394575, 2.1815033 ]), array([0.91237538, 1.91532045]), array([1.27599433, 2.08148045]), array([0.87100384, 1.87033246]), array([0.95154867, 2.07456851]), array([1.33544931, 2.14101885]), array([0.59711942, 1.76697969]), array([1.50237954, 1.88445317]), array([0.50608528, 1.9015923 ]), array([1.0418898 , 2.29771111]), array([0.80653877, 1.89983465]), array([0.38715643, 1.80579165]), array([0.74036687, 1.81878   ]), array([1.19087458, 1.90462095]), array([0.36925039, 1.86141909]), array([1.01630051, 2.06572244]), array([1.24280116, 2.13205854]), array([0.34014098, 1.93839148]), array([1.55665124, 2.21078217]), array([1.10027114, 1.92849439]), array([0.97333465, 1.93477137]), array([0.62664762, 1.962814  ]), array([1.42674607, 2.06530489]), array([0.39463222, 1.96875374]), array([1.73405812, 2.13596805]), array([1.15503007, 1.97117026]), array([-0

In [4]:
# x = []
# y = []
# for i in range(len(sim_Z)):
#   x.append(sim_Z[i][0])
#   y.append(sim_Z[i][1])
#
# plt.scatter(x, y, color = "pink", alpha=.40)
# plt.title('Simulation de Z')

fig1 = figure(title='simulation de Z', width=490, height=300, background_fill_color="#fafafa")
fig1.circle([_[0] for _ in sim_Z], [_[1] for _ in sim_Z], 
            size=5, color='pink', alpha=.40)
fig1.yaxis.axis_label = r'Z₂'
fig1.xaxis.axis_label = r'Z₁'
fig1.grid.grid_line_color="white"

show(fig1)

## l’estimation parametrique
## Question 1

In [5]:
t = np.mat([2,0,1,9]).T
theta = [1,2,1]
n = 4
# print(t)
def sampleX(t, theta):
    X = np.zeros(n)
    for i in range(n):
        X[i] = np.random.normal(theta[0]+theta[1]*t[i][0],np.sqrt(theta[2]))
    return X
  
X = sampleX(t, theta)
print(X)

[ 3.618862   -0.84534622  2.75151175 18.14325765]


## Question 2

In [6]:
# Calculs de estimateur de maximum vraisamblance

# Calculs de bata_1 et beta_2

average_t =np.array( sum(t)/n )
average_t = average_t[0][0]
average_t2 =  np.array(sum(np.dot(t.T,t))/n)
average_t2 = average_t2[0][0]
s = average_t2 - average_t * average_t
matrix_transaction_1 =  np.matrix([ [1/(n*s)*(average_t2), - 1/(n*s)*average_t] , [- 1/(n*s)*average_t,  1/(n*s)]])
# print(matrix_transaction_1)

array_ones = np.array([np.ones(4)])
matrix_transaction_2 = np.concatenate( (array_ones, t.T), axis = 0 )
# print(matrix_transaction_2)

def beta_sigma_est(X):
    bh = np.dot(np.dot(matrix_transaction_1, matrix_transaction_2), X).A1
    sh2 = np.dot(X - bh[0] * array_ones - bh[1] * t.T, (X - bh[0] * array_ones - bh[1] * t.T).T)
    sh2 = sh2.A1[0]/4
    return np.append(bh, sh2)
beta_sigma_estimation = beta_sigma_est(X)
    

Donc on a le resultat de la simulation de $\hat\beta$ et $\hat{\sigma^2}$: 

In [7]:
print(beta_sigma_estimation)
# le resultat est sous la forme de ``[beta_1, beta_2, sigma^2]''

[-0.21935065  2.04547398  0.32856656]


## Question 3

In [8]:
n_ech = 1000  # 1000 echantillons
betas = []
sigmas = []
for i in range(n_ech):
    bbs = beta_sigma_est(sampleX(t, theta))
    betas.append(bbs[0:2])
    sigmas.append(bbs[2])

In [9]:
df = 4-1
sps.chi(2, df)
x = np.linspace(chi.ppf(0.1),chi.ppf(0.9999), 100)
rv = chi.pdf(x)

In [10]:
fig1 = figure(title='simulation de \hat β de maximum vraisamblance', width=490, height=300, background_fill_color="#fafafa")
fig1.circle([_[0] for _ in betas], [_[1] for _ in betas], 
            size=5, color='pink', alpha=.30)
fig1.yaxis.axis_label = r'\hat β₂'
fig1.xaxis.axis_label = r'\hat β₁'
fig1.grid.grid_line_color="white"

fig2 = figure(title='simulation de \hat σ² de maximum vraisamblance', width=490, height=300, background_fill_color="#fafafa")
hist, edges = np.histogram(sigmas, bins=20)
fig2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], 
          fill_color="navy", line_color="white", alpha=0.5, legend='simulation')
fig2.line(x-3, rv*n_ech*0.6, line_color="#ff8888", line_width=4, alpha=0.7, legend="PDF de χ²(n-1) * N * σ²")
fig2.yaxis.axis_label = 'count'
fig2.xaxis.axis_label = '\hat σ²'
fig2.grid.grid_line_color="white"
show(row(fig1, fig2))

# TODO: check the distribution of sample variance of the gaussian vector...

## Question 4

In [11]:
fig3 = figure(title='La relation entre \hat σ² et \hat β₁', width=490, height=300)
fig3.circle([_[0] for _ in betas], sigmas, 
       size=5, color='pink', alpha=.3)
fig3.yaxis.axis_label = r'\hat σ²'
fig3.xaxis.axis_label = r'\hat β₁'

show(fig3)

On va calculer le coefficient de Pearson entre $\hat{\beta_1}$ et $\hat{\sigma^2}$: 

In [12]:
print('(Pearson\'s correlation coefficient, p)= ', 
      sps.pearsonr([_[0] for _ in betas], sigmas))

(Pearson's correlation coefficient, p)=  (-0.025022491961914802, 0.4292855846072825)


Donc les variables ne sont pas correlees.

### Explication

Nous avons $\hat \sigma^2$ est une fonction déterministe de $(I - P)X$ o\`u  on note $\hat \beta_1 1 + \hat \beta_2 t = PX$. $\hat \beta$ est une fonction déterministe de $PX$;

Donc ce que l'on veut démontrer est que $PX $ et $(I - P)X$ sont incorrélés.

On a:

$$\mathbb{E}_{\theta} ( (I - P)XX^T P ) =  \sigma^2 (I-P)P $$

Par le résultat du Question II du DM1 de MAP433, on a alors P est un projecteur sur l'espace engendré par $t$. Donc $(I-P)P = 0_{\mathbb{R}_n}$ Donc $PX$ et $(I-P)X$ sont indépendents.

On en déduit que $\hat \beta$ et $\sigma^2$ sont incorrélés 